In [210]:
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import cadd_io as cdio
import cadd_main as cdmain
import mdutilities_io as mduio
import mymath as Mmath
import my_plot as myplot
import cadddatadump as cddump
import cadd_mesh2 as cdmesh
import cadd_crack_mesh as cdcmesh
import cadd
import itertools
import newpotential as newpot
import tabularpotential as tabpot
import crystallography_oo as croo
import lineplot_oo3 as lpoo
%matplotlib qt
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Preliminaries

## Directories

In [211]:
maindir = '../Tests/CADD_NoDisl_K_Test/'
uidir = maindir + 'User Inputs/'
fidir = maindir + 'Fortran Inputs/'
dumpdir = maindir + 'Dump Files/'
simname = 'cadd_nodisl_k_test'
simtype = 'cadd_nodisl'

In [212]:
size = 'large'
if size == 'small':
    ferings = 5
elif size == 'medium':
    ferings = 14
elif size == 'large':
    ferings = 24
simname += '_' + size

## Unit System

In [213]:
# LJ?

# Create Inputs

In [214]:
r0 = 1.0

## Nodes/Elements

### Create Mesh

In [215]:
padx, pady = 5, 5
lx, ly = 42, 45 # gives roughly 2500 atoms (including pad, interface)

# initial mesh
crackmesh = cdcmesh.CrackMesh(ferings,lx,ly,padx,pady,r0)

# fix outer nodes for K-test
outernodes = crackmesh.outerbox.all_nodes
crackmesh.mesh.nodes.set_node_bc(outernodes,3)

# modify mesh to separate crack faces
crackmesh.modify_mesh()

mesh = crackmesh.mesh
meshnodes = mesh.nodes

### Plot

In [216]:
crackmesh.mesh.write_dump_all('test.6.dump')
sim = cdmain.Simulation('cadd_nodisl','test',readinput=False)
fig = sim.plot_dump_from_file('test.6.dump')

### Nodes

In [217]:
# set z-coordinates to zero
n = meshnodes.xy.shape[0]
z = np.zeros((n,))
posn = np.column_stack((meshnodes.xy,z)) # z-coordinate = 0
nodes = cdmain.Nodes(posn=posn,types=meshnodes.types)

### FE Elements

In [218]:
feelements = cdmain.FEElement(elname='CPE3',mnum=1,
                              connect=mesh.elements_dump) # elements_dump fixes off by 1 indexing issue

## Potential

In [219]:
brittlenew = newpot.gen_std_potential(ductilityindex=0,dimensions=2,r0=1)

In [220]:
rminsmall, rmin, rmax = 0.01, 0.1, 2.0
npoints = 3000
rvec = np.linspace(rmin,rmax,npoints)
rvec = np.insert(rvec,0,rminsmall)
pottable = brittlenew.get_energy_force_table(rvec)

In [221]:
brittlenew.write_file_lammps('brittlenewnorm',np.linspace(0.1,3.0,3000),extend=True)

In [222]:
# Energy plot
# lpoo.my_quick_plot([pottable[:,[0,1]]],axisbounds=[0.5,2,-1,3])

In [223]:
rnnn = np.sqrt(3)*r0 # 2nd nearest neighbor
potential = cdmain.Potential(pname='brittlenew',forcecutoff=rnnn,pottable=pottable)

## Material

### Elastic constants

In [224]:
elasticdict = brittlenew.elasticdict('hex')
brittlemat = croo.Hex.from_potential(brittlenew)
elconst = brittlemat.voigt_plane_strain_stiffness

In [225]:
elconst

array([[ 40.09879362,  13.36626454,   0.        ],
       [ 13.36626454,  40.09879362,   0.        ],
       [  0.        ,   0.        ,  13.36626454]])

In [226]:
material = cdmain.Material(burgers=r0,disldrag=0.0,dislvmax=0.0,elconst=elconst,lannih=0.0,lattice='hex',mass=1.0,
                           mname='material1',rho=brittlenew.rho)

## Misc

In [227]:
misc = cdmain.Misc(increments=100,timestep=0.02,iscrackproblem=True,potstyle='table')

## Interactions

In [228]:
interactions = cdmain.Interactions(table=np.array([[1,1,1]]))

## Neighbors

In [229]:
neighbors = cdmain.Neighbors(checkdisp=True,every=0,delay=5,images=0,Lz=1.0,skin=0.3)

## Damping

In [230]:
gamma = 0.1 # may need to fiddle with this
damping = cdmain.Damping(flag=True,gname='all',gamma=gamma)

## Everything

In [231]:
caddsim = cdmain.Simulation(simtype,simname,uidir,fidir,dumpdir,readinput=False)
caddsim.data = cdmain.CADDData(simtype,nfematerials=1,nodes=nodes,materials=[material],misc=misc,groups=None,
                               potentials=[potential],interactions=interactions,neighbors=neighbors,damping=damping,
                               feelements=[feelements])

## Write

In [232]:
caddsim.write_fortran_all()

# Misc

# Dump Files

In [242]:
fig = caddsim.plot_dump_from_increment(8,tight=False,fignum=1,axisbounds=[-30,30,-30,30]);

In [234]:
fig = caddsim.plot_dump_from_increment(0,tight=False,fignum=2,axisbounds=[-50,50,-50,50]);

# Log Files

## Determine K

In [235]:
brittlekmat = croo.Hex.from_potential(brittlenew)

In [236]:
aold = np.eye(3).astype(int)
anew = np.array([[-1,1,0],[1,1,-2],[1,1,1]])
burgers = np.array([1,0,-1])

In [237]:
Kicgriffith, Kierice = brittlekmat.k_both(aold,anew,burgers,printoption=True)

Crack Plane: [ 1  1 -2]
Slip Plane: [-1  2 -1]
Theta: 59.99999999999999 (degrees)
Phi: 1.2074182697257333e-06 (degrees)
K_ic: 8.443147767084215
K_ie: 13.422972533947563


## Measured KIc

In [247]:
Kbefore = 8.55 + (7-1)*0.1
Kafter = 8.55 + (8-1)*0.1
Kic = (Kbefore+Kafter)/2

## Comparison

In [249]:
Gratio = (Kic/Kicgriffith)**2
print(Gratio)

1.18731751506


In [240]:
# in paper (Rajan, Warner, Curtin, 2016, MSMSE):
# GIc (MS)/GIc (Griffith) = 1.18